In [ ]:
# Install necessary packages
!pip uninstall -y bitsandbytes accelerate
%pip install -U bitsandbytes accelerate
import bitsandbytes as bnb
import accelerate

# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
#enabling import of custom module for colab
! cp /content/gdrive/MyDrive/Util.py . 

In [ ]:
#set up file location here
import sys
sys.path.append('/content/gdrive/MyDrive')

# import libraries
from Util import load_data_JSON, load_model, write_out
import pandas as pd
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [ ]:
#load the data
general_df, prompt_g, correct_g, gtp4_g, Info_g = load_data_JSON('/content/gdrive/MyDrive/general_data.json', 'general')
qa_df, prompt_q, correct_q, gtp4_q, Info_q = load_data_JSON('/content/gdrive/MyDrive/qa_data.json', 'qa')
sum_df, prompt_s, correct_s, gtp4_s, Info_s = load_data_JSON('/content/gdrive/MyDrive/summarization_data.json', 'sum')

In [ ]:
# Access input (API key, etc.)
access = input('Access code?')

In [ ]:
#setup of model
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf", access)

In [ ]:
gen = pipeline("text-generation", model=model,torch_dtype = torch.float16, tokenizer=tokenizer)

In [ ]:
# System prompt to enforce English-only responses
system_prompt = """[INST] <>
You are a helpful assistant. You will answer only in English and avoid using any other languages. Your answers are clear and concise.
<>
"""

In [ ]:
# Function to get model response
def get_response(prompt, max_len):
    sequences = gen(prompt, do_sample=True, top_k=5, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=max_len + len(prompt))
    return sequences[0]['generated_text']

In [ ]:
# ChainPoll method to detect hallucinations
def chainpoll(model, tokenizer, prompts, num_responses=5):
    out_q = []

    for i, prompt in enumerate(prompts[:2]):
        hallucination_scores = []
        prompt_with_system = system_prompt + f"{prompt} [/INST]"

        for _ in range(num_responses):
            # Tokenize the prompt
            inputs = tokenizer(prompt_with_system, return_tensors="pt").to("cuda")

            # Generate the response
            with torch.no_grad():
                outputs = model.generate(
                    inputs['input_ids'],
                    max_length=50,  # Adjust based on needs
                    num_beams=5,
                    early_stopping=True
                )

            # Decode the output response
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Check for hallucinations
            hallucination_prompt = f"Does the following output contain hallucinations? Explain in detail:\n\nOutput: {response}\n"
            hallucination_inputs = tokenizer(hallucination_prompt, return_tensors="pt").to("cuda")

            with torch.no_grad():
                hall_outputs = model.generate(
                    hallucination_inputs['input_ids'],
                    max_length=100,  # Adjust as needed
                    num_beams=5,
                    early_stopping=True
                )

            hallucination_response = tokenizer.decode(hall_outputs[0], skip_special_tokens=True)

            # Check for hallucination in the response
            if "yes" in hallucination_response.lower():
                hallucination_scores.append(1)
            else:
                hallucination_scores.append(0)

        # Calculate final hallucination score
        hallucination_score = sum(hallucination_scores) / num_responses

        # Append the result with formatted output
        out_q.append({
            "prompt": prompt,
            "response": response,
            "hallucination_score": hallucination_score,
            "explanation": hallucination_response
        })

    return out_q

In [ ]:
chainpoll_results = chainpoll(model, tokenizer, prompt_q)

In [ ]:
# Format the output for better readability
formatted_out_q = []
for result in chainpoll_results:
    formatted_output = f"""
    Prompt: {result['prompt']}

    Response: {result['response']}

    Hallucination Score: {result['hallucination_score']}

    Explanation: {result['explanation']}
    """
    formatted_out_q.append(formatted_output)

# Print the formatted results (optional)
for formatted_result in formatted_out_q:
    print(formatted_result)

In [ ]:
out_q = []
for result in chainpoll_results:
    prompt = result["prompt"]
    response = result["response"]
    hallucination_score = result["hallucination_score"]
    explanation = result["explanation"]

    out_q.append({
        "prompt": prompt,
        "response": response,
        "hallucination_score": hallucination_score,
        "explanation": explanation
    })

In [ ]:
# Generate responses using the system prompt
out_q = []
for p in prompt_q:
    prompt = system_prompt + f"{p} [/INST]"
    response = get_response(prompt, 256)
    response = response[len(prompt):]
    out_q.append(response)

In [ ]:
# Write the results
write_out("/content/gdrive/MyDrive/ChainPoll_qa_results.csv", formatted_out_q)